In [1]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

In [4]:
N = 400
feats = 784
X = rng.randn(N, feats)
y = rng.randint(size=N, low=0, high=2)
training_steps = 10000

In [13]:
# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats), name="w")
b = theano.shared(0., name="b")
print("Initial model:")
print(w.get_value())
print(b.get_value())

Initial model:
[-0.77311051  1.13910135  2.4290085  -0.80616543  0.37179973  1.72392896
  0.08934692  0.52876131 -0.42469566 -0.33290664 -0.39152573 -0.07378602
 -1.31126538 -0.22737733 -0.21916049 -0.63803539  1.73985232  0.43324941
  0.29226819  0.22605433 -1.24276103  0.06143624  2.00494978 -0.43425601
  2.43051599  0.55374752  1.46507505 -0.46317398 -0.8788887  -0.72035155
  0.448513   -0.01368043 -0.62268484 -0.08343702  1.41167803  0.70643969
  0.98108908 -0.47489522  1.58814005 -0.79801291  1.98363638 -1.27897655
  1.80208697  1.22016389  2.06438066  1.19618718 -0.19230294 -0.4708798
  1.4977385   0.27332353 -0.28442339 -0.53043341  0.60541415 -0.01283131
 -0.38013245  0.35728715 -0.81578595  1.04779682 -0.8169647  -0.06623868
 -0.73025849 -1.00377086  0.23244069 -0.14929675  1.17958557 -2.50027732
 -0.29521467 -0.03146161  0.98465067  0.61853135 -1.23475526 -0.15641101
  0.4697833   1.37118587  0.36304128  0.9024883   0.14479749 -0.41096398
  1.39856715 -2.74291268  0.46881212 

In [14]:
# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # (we shall return to this in a
                                          # following section of this tutorial)

In [15]:
# Compile
train = theano.function(inputs = [x,y],
                        outputs = [prediction, xent],
                        updates = ((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs = prediction)

In [6]:
# Train
for i in range(training_steps):
    pred, err = train(X, y)

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[ -7.19961209e-02  -1.54529100e-01   8.85333260e-02   1.54052002e-01
  -5.34341636e-02  -1.32020922e-02  -3.04844484e-02  -4.00060569e-02
   9.49840454e-02   2.94855021e-02  -2.19437745e-01  -1.27247863e-01
  -5.68292007e-03   7.08693331e-02  -1.41432096e-01   6.15046968e-02
  -1.04788293e-01  -1.76824958e-01   1.01292077e-01  -9.52279553e-02
  -6.65484838e-02   7.85339652e-03  -4.21859893e-02   4.99423757e-02
   8.01061549e-02   1.57070843e-02  -5.64720323e-02  -1.25112656e-03
   3.18734410e-03  -3.67635922e-02  -9.55293842e-05   1.00278003e-01
  -3.07085593e-02   1.72579672e-01  -1.67291644e-01   2.40286646e-01
  -1.23155874e-02   6.49845982e-02   1.73571141e-01  -2.15621184e-02
   8.84407142e-02   6.36183818e-02   4.25095605e-02  -2.02204057e-01
   8.65377580e-02  -8.11913418e-03   9.62665804e-03   1.52726502e-02
   2.22317927e-02  -1.57678985e-01   7.50791111e-02   1.11356992e-01
   1.23392884e-01   5.84267089e-02   5.65496128e-02  -4.35136662e-02
   1.20542739e-01   5